In [ ]:
from scipy.stats import norm
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
def epanechnikov_kernel(z):
    return (3 / 4) * (1 - z ** 2) * (np.abs(z) < 1)

def gaussian_kernel(z):
    return (2 * math.pi) ** (-1 / 2) * np.exp(z ** 2 / 2)

def triweight_kernel(z):
    return (35 / 32) * (1 - z ** 2) ** 3 * (np.abs(z) < 1)

def plug_in_h(X, delta):
    N = X.shape[0]
    stdev = np.std(X)
    
    return 1.3643 * delta * (N ** (-1 / 5)) * stdev

def silverman_h(X, delta):
    N = X.shape[0]
    stdev = np.std(X)
    iqr = np.percentile(X, 75, interpolation='midpoint') - np.percentile(X, 25, interpolation='midpoint')
    
    return 1.3643 * delta * (N ** (-1 / 5)) * np.min([stdev, iqr / 1.349])

def cross_validation_h(Y, X, kernel):
    N = X.shape[0]
    bandwidth_values = np.arange(0.15, 1, 0.05)
    
    optimal_h = 10000
    CV = 10000
    
    if kernel == 'epanechnikov':
        k = epanechnikov_kernel
    elif kernel == 'gaussian':
        k = gaussian_kernel
    else:
        k = triweight_kernel
        
    iota = np.ones(N)
    differ = np.zeros(N)
    for bandwidth in bandwidth_values:
        for i in range(N):
            weights = np.zeros(N)

            X_0 = X[i]
            Z = (iota * X_0 - X) / bandwidth
            KX = k(Z)

            YKX = Y * KX
            m_regress = np.mean(YKX) / np.mean(KX)

            for j in range(N):
                Z_nom = (X_0 - X[j]) / bandwidth
                nominator = (1 / (N * bandwidth)) * k(Z_nom)

                Z_denom = (iota * X_0 - X) / bandwidth
                denominator = (1 / (N * bandwidth)) * np.sum(k(Z_denom))

                weights[j] = nominator / denominator

            differ[i] = (Y[i] - m_regress) / (1 - (weights[i] / np.sum(weights)))

        lower_X = np.percentile(X, 5, interpolation='midpoint')
        upper_X = np.percentile(X, 95, interpolation='midpoint')

        pi = np.zeros(N)
        for i in range(N):
            pi[i] = int(((X[i] < upper_X) & (X[i] > lower_X)))

        CV_val = np.sum(differ ** 2 * pi)
        
        if CV_val < CV:
            CV = CV_val
            optimal_h = bandwidth
        
        
        
    return optimal_h         

In [ ]:
def npregress(Y, X, kernel, bandwidth, bandwidth_optimal=None, Xmidpoints=None, nrbins = 100):
    
    nrows, ncols = X.shape
    
    YX = np.concatenate((Y, X), axis=1)
    YX = YX[YX[:,1].argsort()]
    
    Y = YX[:, 0]
    X = YX[:, 1]
    
    meanX = np.mean(X)
    stdX = np.std(X)
    iota = np.ones([nrows, 1])
    
    if Xmidpoints == None:
        nrbins = nrbins
        
        lowerX = X[int(np.floor(0.01 * nrows))]
        upperX = X[int(np.floor(0.99 * nrows - 1))]
        
        Xmidpoints_used = np.reshape(np.linspace(lowerX, upperX, nrbins), newshape=(nrbins, 1))
        
    else:
        nrbins, ncbins = Xmidpoints.size
        Xmidpoints_used = Xmidpoints
        
    firstb = Xmidpoints_used[0, :]
    lastb = Xmidpoints_used[nrbins - 1, :]
    binsize = ((lastb - firstb) / nrbins)
    
    if kernel == 'epanechnikov':
        k = epanechnikov_kernel
        delta = 1.7188
    elif kernel == 'gaussian':
        k = gaussian_kernel
        delta = 0.7764
    else:
        k = triweight_kernel
        delta = 2.3122
    
    if bandwidth_optimal == None:
        bandwidth_used = bandwidth
    elif bandwidth_optimal == 'plug_in':
        bandwidth_used = plug_in_h(X, delta)
    elif bandwidth_optimal == 'silverman':
        bandwidth_used = silverman_h(X, delta)
    else:
        bandwidth_used = cross_validation_h(Y, X, kernel)
        
    m_regress = np.zeros(nrbins)
    
    for j in range(nrbins):
        Xb = Xmidpoints_used[j, 0]
        Z = (iota * Xb - X) / bandwidth_used
        KX = k(Z)
        
        YKX = Y * KX
        m_regress[j] = np.mean(YKX) / np.mean(KX)

        
    return Xmidpoints_used, m_regress, bandwidth_used

In [ ]:
def normpdf(x):
    pi = 3.1415926
    denom = (2*pi*np.var(x))**.5
    num = np.exp(-(x-np.mean(x))**2/(2*np.var(x)))
    return num/denom

def generate_data(N):
    
    X = np.random.uniform(low=0, high=30, size=(N, 1))
    u = np.random.normal(loc=0, scale=1, size=(N, 1))
    X = np.sort(X)
    y = (np.sin(X) + 2) * X + u
    
    yX = np.concatenate((y, X), axis=1) 
    
    yX = yX[yX[:, 1].argsort()]
    
    return yX 

## Data generation

In [ ]:
data = generate_data(1000)

#### OLS

In [ ]:
def local_weighted_OLS(dataset, index):
    
    ##### INPUT
    
    ### data 
    # is a Nxk dimensional array, having y in the first column and X in all following columns
    
    ### index
    # index of an observation for which the weighted OLS has to be calculated
    
    ##### OUTPUT
    # 1x1 array (or float) containing local weighted average of the observation 
    
    y = dataset[:, 0]
    X = dataset[:, 1]
    
    N = X.shape[0]
    
    x_0 = X[index]
    
    var_X = np.sum((X - np.mean(X)) ** 2)
    peacewise_var_X = X - np.mean(X)
    
    OLS_est = np.sum(((1 / N) + (x_0 - np.mean(X)) * peacewise_var_X / var_X ) * y)
    
    return OLS_est

In [ ]:
OLS_est = np.zeros([data.shape[0], 2])

for i in range(data.shape[0]):
    
    OLS_est[i, 0] = local_weighted_OLS(data, i)
    OLS_est[i, 1] = data[i, 1]

In [ ]:
plt.plot(data[:, 1], data[:, 0])
plt.plot(OLS_est[:, 1], OLS_est[:, 0])
plt.show()

#### $k$ nearest neighbors

In [ ]:
def generate_data_sinx_x(N):
    
    X = np.random.uniform(low=0, high=30, size=(N, 1))
    u = np.random.normal(loc=0, scale=10, size=(N, 1))
    y = (np.sin(X) + 2) * X + u
    
    yX = np.concatenate((y, X), axis=1) 
    
    yX = yX[yX[:, 1].argsort()]
    
    return yX

def generate_data_expx_x(N):
    
    X = np.random.uniform(low=0, high=8, size=(N, 1))
    u = np.random.normal(loc=0, scale=10, size=(N, 1))
    y = (np.exp(X) + 2) * X + u
    
    yX = np.concatenate((y, X), axis=1) 
    
    yX = yX[yX[:, 1].argsort()]
    
    return yX

def knn_mean (response, no, step):
    if no - step >= 0 & no + step <= response.shape[0]:
        knn_mean_i = np.mean(response[(no - step) : (no + step + 1)])
        return(knn_mean_i)
    else:
        return(knn_mean(response, no, step-1))
            

def knn_regr (dataset, k):
    #print('Be sure that Y is the first column and X is the second column!')
    N = dataset.shape[0]
    target = np.zeros(N)
    data = dataset[dataset[:, 1].argsort()]
    Y = dataset[:, 0]
    X = dataset[:, 1]
                    
    for i in range(N):
        target[i] = knn_mean(Y, i, k)
        
    return(target)

def MSE (target, predicted):
    return(np.sum((target-predicted) ** 2))

def discr_smoothness(series):
    sum_of_jumps = 0
    for i in range(series.shape[0]-1):
        sum_of_jumps += np.abs(series[i+1] - series[i])
        
    return(sum_of_jumps)    
    
def knn_regr_komplit (dataset, init_k, lam):
    data = dataset[dataset[:, 1].argsort()]
    predicted = knn_regr(data, init_k)
    ms_error = MSE(predicted, data[:, 0])    
    jumps = discr_smoothness(predicted)
    penal = np.abs(ms_error + lam * jumps)
           
    return(ms_error, discr_smoothness(predicted), penal, predicted)

def optimal_k (dataset, init_k, lam):
    data = dataset[dataset[:, 1].argsort()]
    a, b, c, d = knn_regr_komplit(data, init_k, lam)
    predicted_init = d
    ms_error_init = a  
    jumps_init = b
    penal_best = c
    best_k = init_k       
    
    for k in np.arange(6, 153, 3):
        penal_new = knn_regr_komplit(data, k, lam)[2]
        if penal_new < penal_best:
            best_k = k
        
    return(best_k)

In [ ]:
data = generate_data_sinx_x(1000)
lam = 40
opt_k = optimal_k(data, 20, lam)        
a, b, c, d = knn_regr_komplit(data, opt_k, lam)
plt.scatter(data[:, 1], data[:, 0], alpha=0.4)
plt.plot(data[:, 1], (np.sin(data[:, 1]) + 2) * data[:, 1], c = 'black')
plt.plot(data[:, 1], d, c = 'red')
plt.show()

print('MSE:', a)
print('Jumps:', b)
print('Penal:', c)
print('For lambda =', lam, ' the optimal k is:', opt_k)

In [ ]:
data = generate_data_expx_x(1000)
lam = 40
opt_k = optimal_k(data, 20, lam)        
a, b, c, d = knn_regr_komplit(data, opt_k, lam)
plt.scatter(data[:, 1], data[:, 0], alpha=0.4)
plt.plot(data[:, 1], (np.exp(data[:, 1]) + 2) * data[:, 1], c = 'black')
plt.plot(data[:, 1], d, c = 'red')
plt.show()

print('MSE:', a)
print('Jumps:', b)
print('Penal:', c)
print('For lambda =', lam, ' the optimal k is:', opt_k)

#### Epanechnikov kernel for regression

In [ ]:
y = np.reshape(data[:, 0], (data[:, 0].shape[0], 1))
X = np.reshape(data[:, 1], (data[:, 1].shape[0], 1))

midpoints, fitted, band = npregress(Y=y, X=X, kernel='epanechnikov', 
                                    bandwidth=0.2, bandwidth_optimal='cross_validation',
                                    nrbins=100)

In [ ]:
plt.scatter(X, y, alpha=0.1)
plt.plot(midpoints, fitted, c='orange')
plt.show()